**Load Libraries**



In [2]:
#Loading required libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import spacy
import string
import gensim
import operator
import re

**Load the data**

In [3]:
# Reading the two datasets
lowes_df = pd.read_csv("lowes.csv")
home_depot_df = pd.read_csv("hd_spec_8.csv")
# Havig a look at top 5 rows of the datasets.
lowes_df.head()
#home_depot_df.head()

,sku,color,channel,department,category,brand,product_id,category_id,department_id,product_name,description,feature_list,style_attributes,pdp_url,feature_image,launch_on,text
0,1000512177,White,Lowes US,Hardware,Curtain Rods,Project Source,fb99a2cbe786dabf37631e567d64a89f511f961ca45522...,[642],[15],Project Source 28-in To 48-in White Steel Sing...,28-in To 48-in White Steel Single Curtain Rod ...,"['Ideal for layered window treatments', 'Mount...","{'Hardware Included': 'Yes', 'Warranty': 'None...",https://www.lowes.com/pd/Project-Source-28-in-...,https://assets.stylumia.com/originals/2022/11/...,2022-11-18,"color is White, brand is Project Source, pro..."
1,1001804078,Gold,Lowes US,Hardware,Curtain Rods,Achim,a3c3790f08e521453526beeda2fe3d5852dc46d7ce87e0...,[642],[15],Achim Innovative Bailey 28-in to 48-in Gold St...,Innovative Bailey 28-in to 48-in Gold Steel Si...,"['Innovative wrap around design', 'All metal r...","{'Hardware Included': 'Yes', 'Warranty': 'None...",https://www.lowes.com/pd/Achim-Innovative-28-i...,https://assets.stylumia.com/originals/2022/11/...,2022-11-18,"color is Gold, brand is Achim, product name ..."
2,1001004130,Antique Brass,Lowes US,Hardware,Curtain Rods,Hart & Harlow,3be5062874474d6dcc8b41fe4d9fcf926fa334f7b26f7f...,[642],[15],Hart & Harlow Hedera 13/16-in 4-sided 28-in To...,Hedera 13/16-in 4-sided 28-in To 48-in Antique...,['Includes four 13/16-in diameter adjustable p...,"{'Hardware Included': 'Yes', 'Color/Finish Fam...",https://www.lowes.com/pd/Hart-Harlow-Hedera-13...,https://assets.stylumia.com/originals/2022/11/...,2022-11-18,"color is Antique Brass, brand is Hart & Harl..."
3,1001006546,Black,Lowes US,Hardware,Curtain Rods,Hart & Harlow,8be40ae7c994e60ae7444e5d7ed2d17498cb73b4610f1d...,[642],[15],Hart & Harlow Cluster 13/16-in Double 28-in to...,Cluster 13/16-in Double 28-in to 48-in Black S...,['Includes four 13/16-in diameter adjustable p...,"{'Hardware Included': 'Yes', 'Color/Finish Fam...",https://www.lowes.com/pd/Hart-Harlow-Cluster-1...,https://assets.stylumia.com/originals/2022/11/...,2022-11-18,"color is Black, brand is Hart & Harlow, prod..."
4,1001003946,Black,Lowes US,Hardware,Curtain Rods,Hart & Harlow,3ff70c3fce0a4af93711f004bb0c7d31f10e200c4cbe6b...,[642],[15],Hart & Harlow Cluster 13/16-in 5-sided 28-in t...,Cluster 13/16-in 5-sided 28-in to 48-in Black ...,['Includes five 13/16-in diameter adjustable p...,"{'Hardware Included': 'Yes', 'Color/Finish Fam...",https://www.lowes.com/pd/Hart-Harlow-Cluster-1...,https://assets.stylumia.com/originals/2022/11/...,2022-11-18,"color is Black, brand is Hart & Harlow, prod..."


In [4]:
#Selecting the most appropriate texual column to work on in Lowes dataset.
Nlowes= lowes_df["text"]
Nlowes

0      color is  White, brand is  Project Source, pro...
1      color is  Gold, brand is  Achim, product name ...
2      color is  Antique Brass, brand is  Hart & Harl...
3      color is  Black, brand is  Hart & Harlow, prod...
4      color is  Black, brand is  Hart & Harlow, prod...
                             ...                        
392    color is  Antique Brass, brand is  Hart & Harl...
393    color is  Black, brand is  Lumino, product nam...
394    color is  Pewter, brand is  Style Selections, ...
395    color is  Dark Oil Rubbed Bronze, brand is  Lu...
396    color is  Beachwood, brand is  Kenney, product...
Name: text, Length: 397, dtype: object

In [5]:
#Combining multiple columns in Home Depot dataset to form a new column named 'text' which contains all the relevant data.
home_depot_df["text"] = home_depot_df[['color','style_attributes','feature_list','description','mrp','brand','product_name','spec','lenght','finish','material']].apply(lambda x: " ".join(str(val) for val in x if not pd.isnull(val)), axis=1)
Nhd = home_depot_df["text"]
Nhd

0       Silver {'Finish Family': 'Satin Nickel', 'Rod/...
1       Brown {'Finish Family': 'Brown', 'Rod/Pole Typ...
2       Silver {'Finish Family': 'Silver', 'Rod/Pole T...
3       Silver {'Finish Family': 'Satin Nickel', 'Rod/...
4       Gold {'Finish Family': 'Gold', 'Rod/Pole Type'...
                              ...                        
2659    Black {'Adjustable Rod': ['Adjustable'], 'Colo...
2660    Brown {'Adjustable Rod': ['Adjustable'], 'Colo...
2661    Gold {'Adjustable Rod': ['Adjustable'], 'Color...
2662    Brown {'Adjustable Rod': ['Adjustable'], 'Colo...
2663    White {'Adjustable Rod': ['Adjustable'], 'Colo...
Name: text, Length: 2664, dtype: object

**Text preprocessing**

In [6]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string


In [7]:
#Lowercasing the data.
Nlowes = Nlowes.str.lower()
Nhd = Nhd.str.lower()
#Nlowes
Nhd

0       silver {'finish family': 'satin nickel', 'rod/...
1       brown {'finish family': 'brown', 'rod/pole typ...
2       silver {'finish family': 'silver', 'rod/pole t...
3       silver {'finish family': 'satin nickel', 'rod/...
4       gold {'finish family': 'gold', 'rod/pole type'...
                              ...                        
2659    black {'adjustable rod': ['adjustable'], 'colo...
2660    brown {'adjustable rod': ['adjustable'], 'colo...
2661    gold {'adjustable rod': ['adjustable'], 'color...
2662    brown {'adjustable rod': ['adjustable'], 'colo...
2663    white {'adjustable rod': ['adjustable'], 'colo...
Name: text, Length: 2664, dtype: object

In [8]:
#Removing Punctuations in data.
string.punctuation
exclude = string.punctuation
Nlowes = Nlowes.str.translate(str.maketrans('', '', exclude))
Nhd = Nhd.str.translate(str.maketrans('', '', exclude))
#print(Nlowes)
print(Nhd)


0       silver finish family satin nickel rodpole type...
1       brown finish family brown rodpole type single ...
2       silver finish family silver rodpole type singl...
3       silver finish family satin nickel rodpole type...
4       gold finish family gold rodpole type double pr...
                              ...                        
2659    black adjustable rod adjustable color family b...
2660    brown adjustable rod adjustable color family b...
2661    gold adjustable rod adjustable color family go...
2662    brown adjustable rod adjustable color family b...
2663    white adjustable rod adjustable color family w...
Name: text, Length: 2664, dtype: object


In [9]:
#Removing Stopwords in data
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

a = Nlowes.apply(remove_stopwords)
b = Nhd.apply(remove_stopwords)
#a
b


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


0       silver finish family satin nickel rodpole type...
1       brown finish family brown rodpole type single ...
2       silver finish family silver rodpole type singl...
3       silver finish family satin nickel rodpole type...
4       gold finish family gold rodpole type double pr...
                              ...                        
2659    black adjustable rod adjustable color family b...
2660    brown adjustable rod adjustable color family b...
2661    gold adjustable rod adjustable color family go...
2662    brown adjustable rod adjustable color family b...
2663    white adjustable rod adjustable color family w...
Name: text, Length: 2664, dtype: object

In [10]:
#Removing duplicates in data.
from collections import Counter

def remove_duplicate(s):
    word_list = s.split(" ")
    word_dict = Counter(word_list)
    result = " ".join(word_dict.keys())
    return result

a_new = a.apply(lambda x: remove_duplicate(x))
b_new = b.apply(lambda x: remove_duplicate(x))
a_new.head()
#b_new.head()

0    color white brand project source product name ...
1    color gold brand achim product name innovative...
2    color antique brass brand hart harlow product ...
3    color black brand hart harlow product name clu...
4    color black brand hart harlow product name clu...
Name: text, dtype: object

In [11]:
#onverting the strings data to a DataFrame
df_a = a_new.to_frame()
df_b = b_new.to_frame()
df_b

,text
0,silver finish family satin nickel rodpole type...
1,brown finish family rodpole type single produc...
2,silver finish family rodpole type single produ...
3,silver finish family satin nickel rodpole type...
4,gold finish family rodpole type double product...
...,...
2659,black adjustable rod color family decor type w...
2660,brown adjustable rod color family drapery hard...
2661,gold adjustable rod color family drapery hardw...
2662,brown adjustable rod color family drapery hard...


In [12]:
#Tokenization and Lemmatization.
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import pandas as pd
nltk.download('punkt')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

# Tokenize and lemmatize the data row-wise
for index, row in df_a.iterrows():
    text = row['text']  # Replace 'text' with the actual column name containing the text data
    tokens = word_tokenize(text)
    lemmas = [lemmatizer.lemmatize(token) for token in tokens]
    df_a.at[index, 'lemmatized_text'] = ' '.join(lemmas)  # Create a new column 'lemmatized_text' to store the lemmatized data


for index, row in df_b.iterrows():
    text = row['text']  # Replace 'text' with the actual column name containing the text data
    tokens = word_tokenize(text)
    lemmas = [lemmatizer.lemmatize(token) for token in tokens]
    df_b.at[index, 'lemmatized_text'] = ' '.join(lemmas)
# Print the updated DataFrame with lemmatized data
print(df_b)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


                                                   text  \
0     silver finish family satin nickel rodpole type...   
1     brown finish family rodpole type single produc...   
2     silver finish family rodpole type single produ...   
3     silver finish family satin nickel rodpole type...   
4     gold finish family rodpole type double product...   
...                                                 ...   
2659  black adjustable rod color family decor type w...   
2660  brown adjustable rod color family drapery hard...   
2661  gold adjustable rod color family drapery hardw...   
2662  brown adjustable rod color family drapery hard...   
2663  white adjustable rod color family drapery hard...   

                                        lemmatized_text  
0     silver finish family satin nickel rodpole type...  
1     brown finish family rodpole type single produc...  
2     silver finish family rodpole type single produ...  
3     silver finish family satin nickel rodpole type...  
4

**Modelling**

In [19]:
#Importing libraries.
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import normalize
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics.pairwise import cosine_similarity


In [20]:
#Combining both the datasets in a single dataset.
combined_data = df_a['lemmatized_text'].tolist() + df_b['lemmatized_text'].tolist()


In [34]:
#Vectorize the data using TF-IDF
vectorizer = TfidfVectorizer()
combined_vectorized = vectorizer.fit_transform(combined_data)

#Vectorize the data using Bag of Words
#vectorizer = CountVectorizer()
#combined_vectorized = vectorizer.fit_transform(combined_data)

#Normalize the vectorized data for BOW
#combined_vectorized_normalized = normalize(combined_vectorized)

In [41]:
k =5 # Number of nearest neighbors to retrieve
knn = NearestNeighbors(n_neighbors=k)
knn.fit(combined_vectorized)

#Knn fit for normalized data.(BOW)
#knn.fit(combined_vectorized_normalized)



NearestNeighbors()

In [17]:
#Using decision tree
# labels = [0] * len(df_a) + [1] * len(df_b)

# Train the Decision Tree classifier
# clf = DecisionTreeClassifier()
# clf.fit(combined_vectorized, labels)

# Find matches using the trained classifier
# df_a_vectorized = vectorizer.transform(df_a['lemmatized_text'])
# predictions = clf.predict(df_a_vectorized)

# Print matched products and confidence scores
# for i, lowes_product in enumerate(df_a['lemmatized_text']):
#     print("Lowes Product:", lowes_product)
#     print("Matched Home Depot Product:", df_b['lemmatized_text'][predictions[i]])
#     print()

In [42]:
#Using TF-IDF
df_a_vectorized = vectorizer.transform(df_a['lemmatized_text'])
similarity_scores, indices = knn.kneighbors(df_a_vectorized)

#Calculating similarity scores using cosine similarity
# df_a_vectorized = vectorizer.transform(df_a['lemmatized_text'])
# similarity_scores = cosine_similarity(df_a_vectorized, combined_vectorized)

#Normalization for BOW
# lowes_vectorized_normalized = normalize(df_a_vectorized)
#similarity_scores, indices = knn.kneighbors(lowes_vectorized_normalized)

#len(similarity_scores)
#len(indices[0])
similarity_scores,indices

(array([[1.49011612e-08, 5.06500344e-01, 5.29502171e-01, 5.51664721e-01,
         5.53405875e-01],
        [0.00000000e+00, 3.87374754e-01, 4.17054740e-01, 7.29923466e-01,
         7.83713922e-01],
        [0.00000000e+00, 5.64668122e-01, 5.90392306e-01, 6.04483585e-01,
         6.18113257e-01],
        ...,
        [2.58095683e-08, 3.81813852e-01, 5.68225245e-01, 5.84971428e-01,
         5.86744663e-01],
        [0.00000000e+00, 8.37317145e-01, 8.42263876e-01, 8.91959759e-01,
         9.08024979e-01],
        [1.49011612e-08, 3.99174425e-01, 5.14343397e-01, 5.59583793e-01,
         5.73613751e-01]]),
 array([[   0,   21,  126,  275,  162],
        [   1,  190,  316,  700, 2143],
        [   2,  324,  304,  339,   60],
        ...,
        [ 394,  211,  128,  108,  236],
        [ 395,  288,   30,  374,   78],
        [ 396,  246,  238,  336,  380]]))

In [43]:

for i, product_a in enumerate(df_a['lemmatized_text']):
    print("Product A:", product_a)

    print("Matched Products from Dataset B:")
    for j in range(len(indices[i])):
        product_b = combined_data[j]
        confidence_score = 1 - similarity_scores[i][j]  # Calculate confidence score (1 - similarity score)
       
        print("Product B:",product_b)
        print("Confidence Score:", confidence_score)
    print()

Streaming output truncated to the last 5000 lines.
Confidence Score: 0.5779620030330656
Product B: color antique brass brand hart harlow product name hedera 1316in 4sided 28in 48in steel bay window curtain rod description dedicated supplying highest quality decorative drapery hardware designer create one kind current style fashion offer variety decorating solution basic elegant specialty accessory installation option provide complete opportunity featurelist includes four diameter adjustable pole eight bracket three corner connector pair finial mounting finial measurement 4in w x 5in h 14in bracket projection 2875 inch clearance 2 side 2848 material rodfinials styleattributes included yes colorfinish family copper use location indoor manufacturer item number 1463758 type inch collection maximum length 48 minimum 28 range unspsc 52131700 moderncontemporary theme na package quantity 1 08125 ca resident prop 65 warning
Confidence Score: 0.5662668736951899
Product B: color black brand hart 